In [ ]:
import os

# 작업 위치 설정
WORKSPACE_DIR = '../'

# data 위치 설정
data_path = os.path.join(WORKSPACE_DIR, 'Data')

from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [ ]:
# helper (디렉토리 생성 함수)
def make_dir(dirname):
    try:
        if not (os.path.isdir(dirname)):
            os.makedirs(dirname)
    except OSError:
        print(f"Failed Create Your Directory : {dirname}")

In [ ]:
from denoising_diffusion_pytorch import Unet, GaussianDiffusion
from trainer import Trainer

model = Unet(
    dim = 64,
    dim_mults = (1, 2, 4, 8)
)

diffusion = GaussianDiffusion(
    model,
    image_size = 32,
    timesteps = 1000,           # p 계산 횟수
    sampling_timesteps = 256    # q 계산 횟수
)

dataset_name = 'cars_shapenet'

In [ ]:
# model 저장 위치 설정
res_path = os.path.join(WORKSPACE_DIR, 'diffusion_model', dataset_name, 'train', 'images')
print(res_path)

In [ ]:
make_dir(res_path)

trainer = Trainer(
    diffusion,
    data_path + '/cars_train_test',   # 사용할 데이터 위치
    dataset_name = dataset_name,
    train_batch_size = 32,
    train_lr = 8e-5,                  # 러닝 레이트
    train_num_steps = 700000,         # 총 training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True,                       # turn on mixed precision
    calculate_fid = False,            # training 중 FID score 산출 여부
    results_folder = res_path,
    gpu_index = 1                     # 사용할 gpu 번호
)

In [ ]:
trainer.train() #학습

In [ ]:
trainer.plot_loss() # loss 출력

In [ ]:
trainer.sample_images_at_milestone(700)